In [1]:
import json
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import os
#from kss import split_sentences

In [2]:
def psg_sent_vec(psg, sents, model):
    vec1 = model.encode(psg)
    vec2 = model.encode(sents)
    return vec1, vec2

def knn(index_vec, test_vec):
    knn_model = NearestNeighbors(n_neighbors=3,
                     metric='cosine',
                     algorithm='brute',
                     n_jobs=-1)
    knn_model.fit(index_vec)
    res = knn_model.kneighbors(test_vec, 20, return_distance=True)
    
    return res

def acc_ch(psg_data, sent_data, psg_lab, model, model_name):
    topn = [1,3,5,10,20]
    psg_vec, sents_vec = psg_sent_vec(psg_data, sent_data, model)
    res = knn(psg_vec, sents_vec)
    
    for tn in topn:
        ac = 0
        for cnt, i in enumerate(res[1]):
            psg_lst = []
            for ps_idx in i[:tn]:
                psg_lst.append(psg_data[ps_idx])
            if psg_lab[cnt] in psg_lst:
                ac+=1
        print(model_name,' ',str(tn), ': ', ac/len(psg_lab))

In [3]:
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [4]:
with open('korquad_test.json', 'r', encoding='utf8') as fp:
    test_data = json.load(fp)

In [5]:
psg_lab = [i['doc'] for i in test_data]
psg = list(set(psg_lab))
sents = [i['sent'] for i in test_data]
psg_st_lab = ['[PSG] ' +i['doc'] for i in test_data]
psg_st = list(set(psg_st_lab))
sents_st = ['[SENT] '+i['sent'] for i in test_data]

In [12]:
acc_ch(psg_st, sents_st, psg_st_lab,ours_curr, 'ours_curr')

ours_curr   1 :  0.6084170419120194
ours_curr   3 :  0.7880152407343263
ours_curr   5 :  0.8458607551091099
ours_curr   10 :  0.9031866989954971
ours_curr   20 :  0.9473501905091791


In [19]:
acc_ch(psg_st, sents_st, psg_st_lab,ours_curr, 'ours_curr')

ours_curr   1 :  0.600969864911673
ours_curr   3 :  0.785417388292345
ours_curr   5 :  0.8455143747835123
ours_curr   10 :  0.9021475580187045
ours_curr   20 :  0.9437131970904052
